In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import locale
import os
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' )

'en_US.UTF-8'

In [18]:
#add in replace nan with 0 for cleaning fee 
def read_in_airbnb_file_pair(files, desired_col):
    both_yrs = []
    for file in files:
        yr = pd.read_csv('./Data/NYC_full_listings/'+file).replace({"Brooklyn" :"Kings", "Manhattan":"New York"})
        yr = yr.dropna(subset = ['id','neighbourhood_cleansed','bedrooms', 'accommodates','price', 'guests_included','extra_people', 'minimum_nights', 'availability_90', 'availability_365'])
        yr = fix_columns(yr.loc[:, desired_col], desired_col)
        yr = yr.loc[yr["room_type"] == "Entire home/apt"]
        both_yrs.append(yr)
    
    return both_yrs[1], both_yrs[0]

In [19]:
#our handpicked initial columns that we want to look at

desired_col=np.array(['id','neighbourhood_cleansed','neighbourhood_group_cleansed', 'property_type', 'room_type', 'accommodates','bathrooms', 'bedrooms', 'beds', 'bed_type', 'price','cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights', 'availability_90','availability_365','review_scores_rating', 'review_scores_accuracy','review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month', 'longitude', 'latitude'])


In [39]:
#get airbnb date pairs for 1 year difference 

airbnb_files = sorted(os.listdir('./Data/NYC_full_listings'))
airbnb_files_all = airbnb_files.remove(".DS_Store")                                ###[MHSJ]
airbnb_files_monthyear = []
for i in range(len(airbnb_files)):
    if airbnb_files[i] == ".DS_Store" or airbnb_files[i] == ".ipynb_checkpoints":  ###[MHSJ]
        continue                                                                   ###[MHSJ]
    year = int(airbnb_files[i].split('_')[0])
    month = (airbnb_files[i].split('_')[1][:-4])            ###[mhsj] is there a reason year is turned into an int but month is not?
    airbnb_files_monthyear.append((month, year))

airbnb_files_datePairs = []
for i,date in enumerate(airbnb_files_monthyear):
    if (date[0], date[1]+1) in airbnb_files_monthyear:
        index_next_yr = airbnb_files_monthyear.index((date[0], date[1]+1))
        airbnb_files_datePairs.append((airbnb_files[i], airbnb_files[index_next_yr]))
print(len(airbnb_files_datePairs))
print(airbnb_files_datePairs[0:5])


27
[('2015_01.csv', '2016_01.csv'), ('2015_05.csv', '2016_05.csv'), ('2015_06.csv', '2016_06.csv'), ('2015_08.csv', '2016_08.csv'), ('2015_09.csv', '2016_09.csv')]


In [50]:

airbnb_files = sorted(os.listdir('./Data/NYC_full_listings'))
airbnb_files_all = airbnb_files.remove(".DS_Store")                                ###[MHSJ]
airbnb_files_monthyear = []
for i in range(len(airbnb_files)):
    if airbnb_files[i] == ".DS_Store" or airbnb_files[i] == ".ipynb_checkpoints":  ###[MHSJ]
        continue                                                                   ###[MHSJ]
    year = int(airbnb_files[i].split('_')[0])            
    month = int(airbnb_files[i].split('_')[1][:-4])
    airbnb_files_monthyear.append((month, year))

#print(airbnb_files_monthyear)

[(1, 2015), (3, 2015), (5, 2015), (6, 2015), (8, 2015), (9, 2015), (10, 2015), (11, 2015), (12, 2015), (1, 2016), (2, 2016), (4, 2016), (5, 2016), (6, 2016), (7, 2016), (8, 2016), (9, 2016), (10, 2016), (11, 2016), (12, 2016), (1, 2017), (2, 2017), (3, 2017), (4, 2017), (5, 2017), (6, 2017), (7, 2017), (8, 2017), (10, 2017), (11, 2017), (12, 2017), (1, 2018), (2, 2018), (3, 2018), (4, 2018), (5, 2018), (6, 2018), (7, 2018), (8, 2018), (9, 2018), (10, 2018)]


In [63]:
###[mhsj] 1 - Add this
#get airbnb date pairs for < 1 year difference 

def get_date_pairs(lag, files, monthyear_files):
    airbnb_files_datePairs = []
    for i,date in enumerate(monthyear_files):
        if date[0]+lag > 12:
            (M,Y) = ((date[0]+lag)%12, date[1]+1)
        else:
            (M,Y) = (date[0] + lag, date[1])
        if (M,Y) in monthyear_files:
            index_next_yr = monthyear_files.index((M,Y))
            airbnb_files_datePairs.append((files[i], files[index_next_yr]))
    return airbnb_files_datePairs

lag_pairs = get_date_pairs(3, airbnb_files, airbnb_files_monthyear)
print(len(lag_pairs))

35


In [21]:
### For 1 year dime difference

airbnb_files_datePairs = []
for i,date in enumerate(airbnb_files_monthyear):
    #print(date[0],date[1]+1)
    if (date[0], date[1]+1) in airbnb_files_monthyear:
        index_next_yr = airbnb_files_monthyear.index((date[0], date[1]+1))
        airbnb_files_datePairs.append((airbnb_files[i], airbnb_files[index_next_yr]))
print(len(airbnb_files_datePairs))
#print(airbnb_files_datePairs[3])

27


In [27]:
def fix_columns(df, desired_col):
    for col in desired_col:
        #print(col)
        df_col = df.loc[:, col]
        if df_col.dtype == 'object':
            if len(df_col) > 0:
                #print(col, df_col.iloc[0])                     
                if np.sum([x[:1] == "$" for x in df_col.iloc[:10]]) > 0:
                    df_col = df_col.fillna('$0.00')
                    new_col = [locale.atof(x[1:]) for x in df_col]
                    df.loc[:,col] = new_col
                elif df_col.iloc[0] == 'f' or df_col.iloc[0] == 't':
                    new_col = [(x=='t') for x in df_col]
                    df.loc[:,col] = new_col
        if col == "neighbourhood_cleansed" or col == "neighbourhood_group_cleansed":
            new_col = [str(x).lower() for x in df_col]
            df.loc[:,col] = new_col
    return df
def fix_zillow_regionname_countyname(df):
    for x in ["RegionName", "CountyName"]:
        df_col = df.loc[:,x]
        new_col = [str(x).lower() for x in df_col]
        df.loc[:,x] = new_col
    return df

#goes through all zillow data and airbnb dataframe for date pair, finds neighborhoods (neighborhood + county pair)
#that are in both data sets and counts how many listings for that neighborhood for each # bedrooms between 0 and 2
#returns list with tuple, where tuple contains matching neighborhood/county sets and a list of bools per bedroom number checking if there are enough listings 
def find_common_neigh(zillow_df, airbnb_df_prev, airbnb_df_curr):
    both_neighs = []
    airbnb_set_prev = set([(x,y) for x,y in zip(airbnb_df_prev["neighbourhood_cleansed"], airbnb_df_prev["neighbourhood_group_cleansed"])])
    airbnb_set_curr = set([(x,y) for x,y in zip(airbnb_df_curr["neighbourhood_cleansed"], airbnb_df_curr["neighbourhood_group_cleansed"])])
    zillow_set = set([(x,y) for x,y in zip(zillow_df["RegionName"],zillow_df["CountyName"])])
    for z, z_c in zillow_set:
        for a, a_c in airbnb_set_curr:
            if z.lower() == a.lower() and (z_c.lower() == a_c.lower() or a_c.lower() == 'nan'):
                for a_p, a_p_c in airbnb_set_prev:
                    if z.lower() == a_p.lower() and (z_c.lower() == a_p_c.lower() or a_p_c.lower() == 'nan'):
                        enough_pts_prev = [len(airbnb_df_prev.loc[(airbnb_df_prev['bedrooms'] == x)&(airbnb_df_prev['neighbourhood_cleansed'] == a)])>30 for x in range(3)]
                        enough_pts_curr = [len(airbnb_df_curr.loc[(airbnb_df_curr['bedrooms'] == x)&(airbnb_df_curr['neighbourhood_cleansed'] == a)])>30 for x in range(3)]
                        enough_pts = [(x&y) for x,y in zip(enough_pts_prev,enough_pts_curr)]
                        both_neighs.append((a.lower(), z_c.lower(), enough_pts))
    return both_neighs

def get_airbnb_county_from_lon_lat(df):
    pass

In [30]:
yr,prev_yr = read_in_airbnb_file_pair(airbnb_files_datePairs[3], desired_col)
print(set(yr.loc[:,"neighbourhood_group_cleansed"]))

print(set(yr.loc[:,"room_type"]))

/Users/Kurtio/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
/Users/Kurtio/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2901: DtypeWarning: Columns (43,88) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


TypeError: 'float' object is not subscriptable

In [65]:
###[MHSJ] function to read in all Airbnb files

def read_in_airbnb_file_all(files, desired_col):
    all_yrs = []
    for file in files:
        yr = pd.read_csv('./Data/NYC_full_listings/'+file, low_memory=False).replace({"Brooklyn" :"Kings", "Manhattan":"New York"})
        yr = yr.dropna(subset = ['id','neighbourhood_cleansed','bedrooms', 'accommodates','price', 'guests_included','extra_people', 'minimum_nights', 'availability_90', 'availability_365'])
        yr = fix_columns(yr.loc[:, desired_col], desired_col)              ###[mhsj] This line is causing problems so I ommited it for now
        yr = yr.loc[yr["room_type"] == "Entire home/apt"]
        all_yrs.append(yr)    
    return all_yrs

In [66]:
###[MHSJ] Read in all airbnb files

airbnb_files_all = sorted(os.listdir('./Data/NYC_full_listings'))
airbnb_files_all.remove(".DS_Store")
all_years = read_in_airbnb_file_all(airbnb_files_all, desired_col)


/Users/Kurtio/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


TypeError: 'float' object is not subscriptable

In [35]:
###[mhsj] same function as above, without the desired cols sections
def read_in_airbnb_file_all(files):
    all_yrs = []
    for file in files:
        yr = pd.read_csv('./Data/NYC_full_listings/'+file, low_memory=False).replace({"Brooklyn" :"Kings", "Manhattan":"New York"})
        yr = yr.dropna(subset = ['id','neighbourhood_cleansed','bedrooms', 'accommodates','price', 'guests_included','extra_people', 'minimum_nights', 'availability_90', 'availability_365'])
        yr = yr.loc[yr["room_type"] == "Entire home/apt"]
        all_yrs.append(yr)    
    return all_yrs

airbnb_files_all = sorted(os.listdir('./Data/NYC_full_listings'))
airbnb_files_all.remove(".DS_Store")
all_years = read_in_airbnb_file_all(airbnb_files_all)


In [28]:
###[mhsj] Look at cols in airbnb files --- can delete after change is made
for i, year in enumerate(all_years):
    for col_name in desired_col:
        if col_name not in list(year.columns):
            print("missing: %s, %s" %(col_name,airbnb_files_all[i]))

missing: cleaning_fee, 2015_01.csv
missing: cleaning_fee, 2015_03.csv
missing: cleaning_fee, 2015_05.csv


In [5]:
###[MHSJ] get unique neighborhood names in airbnb data
list_of_neighbourhoods_unique = []
for df in all_years:
    list_of_names = list(df.neighbourhood_cleansed.unique())
    [list_of_neighbourhoods_unique.append(neighbourhood) for neighbourhood in list_of_names if neighbourhood not in list_of_neighbourhoods_unique]    
#print("Final list airbnb neighbourhoods:", list_of_neighbourhoods_unique)


In [28]:
###[mhsj] # read in zillow data
zillow_files = pd.read_csv("Data/zillow_0_1_2_combined.csv")
zillow_files = fix_zillow_regionname_countyname(zillow_files)
zillow_columns = list(zillow_files.columns)


In [29]:
###[mhsj] list of airbnb neighborhoods that don't appear in list of zillow regions

#get list of zillow regions
list_of_regions_unique_zillow = []
list_of_regions = list(zillow_files.RegionName.unique())
[list_of_regions_unique_zillow.append(region) for region in list_of_regions if region not in list_of_regions_unique_zillow]    
 
#get list of airbnb neighborhoods    
list_of_neighbourhoods_unique = []
for df in all_years:
    list_of_names = list(df.neighbourhood_cleansed.unique())
    [list_of_neighbourhoods_unique.append(neighbourhood.lower()) for neighbourhood in list_of_names if neighbourhood.lower() not in list_of_neighbourhoods_unique]

orphan_neighs = list_of_neighbourhoods_unique.copy()
for region in list_of_regions_unique_zillow:
    if region in orphan_neighs:
        orphan_neighs.remove(region)
orphan_neighs.sort()
print(len(orphan_neighs))


NameError: name 'all_years' is not defined

In [53]:
###[mhsj] Count number of listings in orphan neighborhoods

lost_neighbourhoods = {}
for neighbourhood in orphan_neighs:
    lost_neighbourhoods[neighbourhood] = 0

for year in all_years:
    for neighbourhood in orphan_neighs:
        neigh = [x.lower() for x in year.loc[:,'neighbourhood_cleansed']]
        cnt = neigh.count(neighbourhood)
        #orphan_year = year.loc[year['neighbourhood_cleansed'] == neighbourhood]
        lost_neighbourhoods[neighbourhood] += cnt
        #print(neighbourhood)
        #print(lost_neighbourhoods[neighbourhood])

#print(len(lost_neighbourhoods))


allerton
0
alphabet city
460
arden heights
0
arrochar
0
arverne
0
battery park city
81
bay terrace
0
bay terrace, staten island
0
baychester
7
bayswater
0
bedford-stuyvesant
632
belle harbor
0
bellerose
0
bergen beach
1
breezy point
0
bronxdale
2
brooklyn navy yard
30
brownsville
0
bull's head
0
cambria heights
0
castle hill
0
castle hill 
1
castleton corners
0
charleston
0
chelsea, staten island
0
chinatown
159
city island
4
civic center
21
claremont
2
claremont village
0
clason point
0
clifton
1
co-op city
0
columbia st
0
columbia street waterfront
2
concord
2
concourse village
3
country club
1
cypress hills
0
ditmars / steinway
42
ditmars steinway
0
dongan hills
1
douglaston
0
downtown brooklyn
93
dumbo
27
east elmhurst
1
east morrisania
0
eastchester
0
edenwald
1
edgemere
0
elm park
2
eltingville
0
emerson hill
0
far rockaway
0
fieldston
0
flatlands
5
flushing meadows corona park
0
forest park
0
fort wadsworth
0
fresh meadows
3
gerritsen beach
0
glen oaks
0
gramercy park
188
granit

In [30]:
list_of_regions_unique_zillow.sort()
for N in list_of_regions_unique_zillow:
    print(N)

astoria
auburndale
bath beach
battery park
bay ridge
bayside
bedford park
bedford stuyvesant
belmont
bensonhurst
bergen-lafayette
boerum hill
borough park
briarwood
brighton beach
brooklyn heights
bushwick
canarsie
carnegie hill
carroll gardens
central park
chelsea
clinton hill
cobble hill
college point
concourse
coney island
corona
crown heights
downtown
dyker heights
east flatbush
east harlem
east new york
east village
elmhurst
financial district
flatbush
flatiron district
flushing
fordham
forest hills
fort greene
fort hamilton
garment district
glendale
gowanus
gramercy
gravesend
greenpoint
greenville
greenwich village
greenwood
hamilton heights
harlem
hauppauge
high bridge
homecrest
hunters point
inwood
jackson heights
jamaica
jamaica estates
journal square
kensington
kew gardens
kew gardens hills
kingsbridge
little italy
lower east side
maspeth
mcginley square
middle village
midtown
midwood
morningside heights
morris heights
morris park
murray hill
noho
norwood
oakland gardens
ocea

In [25]:
for neigh in lost_neighbourhoods:
    if lost_neighbourhoods[neigh] > 1000:
        print(neigh, lost_neighbourhoods[neigh])

NameError: name 'lost_neighbourhoods' is not defined

In [32]:
zillow_neighbourhood_dict = {
    'battery park city':'battery park',
    'bedford-stuyvesant':'bedford stuyvesant',
    'chinatown':'lower east side',
    'civic center ':'lower east side',
    'columbia st':'lower east side',
    'cypress hills':'east new york',
    'ditmars steinway':'astoria',
    'downtown brooklyn':'brooklyn heights', 
    'east elmhurst':'jackson heights',
    "hell's kitchen":'midtown',
    "kips bay":'midtown',
    'long island city':'sunnyside',
    'nolita':'soho',
    'ozone park':'south ozone park',
    'prospect-lefferts gardens':'prospect lefferts gardens',
    'red hook':'carroll gardens',
    'south slope':'park slope',
    'stuyvesant town':'midtown',
    'theater district':'midtown',
    'throgs neck':'throggs neck',
    'two bridges':'lower east side',
}

for neigh in zillow_neighbourhood_dict:
    print(neigh,":",zillow_neighbourhood_dict[neigh])


battery park city : battery park
bedford-stuyvesant : bedford stuyvesant
chinatown : lower east side
civic center  : lower east side
columbia st : lower east side
cypress hills : east new york
ditmars steinway : astoria
downtown brooklyn : brooklyn heights
east elmhurst : jackson heights
hell's kitchen : midtown
kips bay : midtown
long island city : sunnyside
nolita : soho
ozone park : south ozone park
prospect-lefferts gardens : prospect lefferts gardens
red hook : carroll gardens
south slope : park slope
stuyvesant town : midtown
theater district : midtown
throgs neck : throggs neck
two bridges : lower east side


## still orphaned neighs


    'brownsville':501,
    'vinegar hill':644, 
    'richmond hill':635,
    'rockaway beach':949,
    'roosevelt island':648,
    'mott haven':628,
    'flatlands':783,
    'dumbo':756,
    'arverne':944,


In [222]:
zillow_files = pd.read_csv("Data/zillow_0_1_2_combined.csv")
zillow_files = fix_zillow_regionname_countyname(zillow_files)
#test_neighs = find_common_neigh(zillow_files, prev_yr, yr)                ###[mhsj] uncomment
test_neighs = find_common_neigh(zillow_files, all_years[1], all_years[2])  ###[mhsj] delete
print(np.sum([np.sum(neigh[2]) for neigh in test_neighs]))
zillow_columns = list(zillow_files.columns)
zillow_columns_fix = {}
for x in zillow_columns:
    new_x = x.replace('-', '_').replace('NumBeds', 'bedrooms')
    zillow_columns_fix[x] = new_x
zillow_files = zillow_files.rename(columns = zillow_columns_fix)
print(zillow_file
    s.columns)


74
Index(['Unnamed: 0', '2010_02', '2010_03', '2010_04', '2010_05', '2010_06',
       '2010_07', '2010_08', '2010_09', '2010_10',
       ...
       '2018_07', '2018_08', '2018_09', 'City', 'CountyName', 'Metro',
       'bedrooms', 'RegionName', 'SizeRank', 'State'],
      dtype='object', length=112)


In [137]:
desired_col_aggregate_quants =np.array(['beds', 'minimum_nights', 'availability_90','availability_365','review_scores_rating', 'review_scores_accuracy','review_scores_location', 'review_scores_value', 'calculated_host_listings_count', 'reviews_per_month'])




In [128]:
def get_num_same_listings(prev_yr, yr):
    pass
def get_rel_increase_in_listings(prev_yr, yr):
    pass
def get_rel_increase_in_avg_feats(prev_yr,yr,feats):
    pass
def get_rel_increase_in_adjprice(prev_yr,yr):
    pass

def create_labeled_set_regression(date_pairs, zillow_files, desired_col, desired_col_aggregate_quants, num_months_predict):
    all_pts = []
    for pair in date_pairs:
        prev_date = pair[0][:-4].split('_')
        date = pair[1][:-4].split('_')
        z_prev_date = [int(x) for x in prev_date]
        z_prev_date = [z_prev_date[0]+int((z_prev_date[1]+2)/12),(z_prev_date[1]+2)%12 + 1]
        z_date = [z_prev_date[0]+1, z_prev_date[1]]
        date = str(z_date[0])+"_"+str(z_date[1]).zfill(2)
        prev_date = str(z_prev_date[0])+"_"+str(z_prev_date[1]).zfill(2)
        print(date)
        if date in list(zillow_files.columns):
            print('found')
            yr, prev_yr = read_in_airbnb_file_pair(pair, zillow_files, desired_col)
            both_neighs = find_common_neigh(zillow_files, prev_yr, yr)
            for neigh in both_neighs:
                for bedrooms, enough in enumerate(neigh[2]):
                    if enough:
                        zillow_files_bd_neigh = zillow_files.loc[(zillow_files["bedrooms"]==bedrooms)&(zillow_files["RegionName"]==neigh[0])&(zillow_files["CountyName"]==neigh[1])]
                        zillow_files_bd_neigh = zillow_files_bd_neigh.loc[:, [prev_date, date]]
                        print(zillow_files_bd_neigh)
                        if len(zillow_files_bd_neigh) > 0:
                            enough_zillow_data = [x != 0 for x in zillow_files_bd_neigh.iloc[0]]
                            print(enough_zillow_data)
                            if np.sum(enough_zillow_data) >= 2:
                                print(np.sum(enough_zillow_data))
                                curr_features = [bedrooms]
                                yr_bd_neigh = yr.loc[(yr['bedrooms'] == bedrooms)&(yr['neighbourhood_cleansed'] == neigh[0])&(yr['neighbourhood_group_cleansed'] == neigh[1])]
                                prev_yr_bd_neigh = prev_yr.loc[(prev_yr['bedrooms'] == bedrooms)&(prev_yr['neighbourhood_cleansed'] == neigh[0])&(prev_yr['neighbourhood_group_cleansed'] == neigh[1])]





In [129]:
create_labeled_set_regression(airbnb_files_datePairs[-6:],zillow_files,desired_col,desired_col_aggregate_quants)

TypeError: create_labeled_set_regression() missing 1 required positional argument: 'num_months_predict'